# Join Statements - Lab

## Introduction

In this lab, you'll practice your knowledge of `JOIN` statements, using various types of joins and various methods for specifying the links between them.

## Objectives

You will be able to:

* Write SQL queries that make use of various types of joins
* Compare and contrast the various types of joins
* Discuss how primary and foreign keys are used in SQL
* Decide and perform whichever type of join is best for retrieving desired data

## CRM ERD

In this lab, you'll use the same customer relationship management (CRM) database that you saw from the previous lesson.
<img src='https://curriculum-content.s3.amazonaws.com/data-science/images/Database-Schema.png' width="600">

## Connecting to the Database
Import the necessary packages and connect to the database `'data.sqlite'`.

In [2]:
import sqlite3
import pandas as pd
conn = sqlite3.connect('data.sqlite')

## Select the names of all employees in Boston 

Hint: join the employees and offices tables. Select the first and last name.

In [4]:
q = """
SELECT firstName, lastName
FROM employees
JOIN offices
    USING(officeCode)
WHERE city = 'Boston'
;
"""
pd.read_sql(q, conn)

,firstName,lastName
0,Julie,Firrelli
1,Steve,Patterson


## Are there any offices that have zero employees?
Hint: Combine the employees and offices tables and use a group by. Select the office code, city, and number of employees.

In [16]:
q = """
SELECT officeCode, city, COUNT(employeeNumber) AS employeeCount
FROM offices
LEFT JOIN employees
    USING(officeCode)
GROUP BY officeCode
HAVING employeeCount = 0
;
"""
pd.read_sql(q, conn)

,officeCode,city,employeeCount
0,27,Boston,0


## Write 3 questions of your own and answer them

In [ ]:
# Answers will vary

# Example question: 
"""
How many customers are there per office?
"""

In [20]:
"""
Question 1

How many customers are there per office?
"""
q = """
SELECT
    officeCode,
    offices.city,
    COUNT(customerNumber) AS n_customers
FROM offices
JOIN employees 
    USING(officeCode)
JOIN customers 
    ON employeeNumber = salesRepEmployeeNumber
GROUP BY officeCode
;
"""
pd.read_sql(q, conn)

# Your code here

,officeCode,city,n_customers
0,1,San Francisco,12
1,2,Boston,12
2,3,NYC,15
3,4,Paris,29
4,5,Tokyo,5
5,6,Sydney,10
6,7,London,17


In [26]:
"""
Question 2

How much in total has each customer paid?
"""

q = """
SELECT customerName, SUM(amount) as total_paid
FROM customers JOIN payments USING (customerNumber)
GROUP BY customerName
;
"""
pd.read_sql(q, conn)

,customerName,total_paid
0,"AV Stores, Co.",148410.09
1,Alpha Cognac,60483.36
2,Amica Models & Co.,82223.23
3,"Anna's Decorations, Ltd",137034.22
4,Atelier graphique,22314.36
...,...,...
93,"Vida Sport, Ltd",108777.92
94,Vitachrome Inc.,72497.64
95,"Volvo Model Replicas, Co",43680.65
96,West Coast Collectables Co.,43748.72


In [28]:
"""
Question 3

How many products has each customer bought?
"""

q = """
SELECT customerName, SUM(quantityOrdered) as total_orders
FROM customers JOIN orders USING (customerNumber) JOIN orderdetails USING (orderNumber)
GROUP BY customerName
;
"""
pd.read_sql(q, conn)

,customerName,total_orders
0,"AV Stores, Co.",1778
1,Alpha Cognac,687
2,Amica Models & Co.,843
3,"Anna's Decorations, Ltd",1469
4,Atelier graphique,270
...,...,...
93,"Vida Sport, Ltd",1078
94,Vitachrome Inc.,787
95,"Volvo Model Replicas, Co",647
96,West Coast Collectables Co.,511


## Level Up 1: Display the names of every individual product that each employee has sold

Hint: You will need to use multiple `JOIN` clauses to connect all the way from employee names to product names.

In [33]:
q = """
SELECT firstName, lastName, productName
FROM employees JOIN customers
    ON employeeNumber = salesRepEmployeeNumber
JOIN orders USING(customerNumber)
JOIN orderdetails USING(orderNumber)
JOIN products USING(productCode)
;
"""
df = pd.read_sql(q, conn)
df

,firstName,lastName,productName
0,Leslie,Jennings,1958 Setra Bus
1,Leslie,Jennings,1940 Ford Pickup Truck
2,Leslie,Jennings,1939 Cadillac Limousine
3,Leslie,Jennings,1996 Peterbilt 379 Stake Bed with Outrigger
4,Leslie,Jennings,1968 Ford Mustang
...,...,...,...
2991,Martin,Gerard,1954 Greyhound Scenicruiser
2992,Martin,Gerard,1950's Chicago Surface Lines Streetcar
2993,Martin,Gerard,Diamond T620 Semi-Skirted Tanker
2994,Martin,Gerard,1911 Ford Town Car


## Level Up 2: Display the number of products each employee has sold

Alphabetize the results by employee last name.

Hint: Use the `quantityOrdered` column from `orderDetails`. Also, think about how to group the data when some employees might have the same first or last name.

In [37]:
q = """
SELECT firstName, lastName, SUM(quantityOrdered) as total_products_sold
FROM employees JOIN customers
    ON employeeNumber = salesRepEmployeeNumber
JOIN orders
    USING(customerNumber)
JOIN orderdetails
    USING(orderNumber)
GROUP BY firstName, lastName
ORDER BY lastName
;
"""
pd.read_sql(q, conn)

,firstName,lastName,total_products_sold
0,Loui,Bondur,6186
1,Larry,Bott,8205
2,Pamela,Castillo,9290
3,Julie,Firrelli,4227
4,Andy,Fixter,6246
5,Martin,Gerard,4180
6,Gerard,Hernandez,14231
7,Leslie,Jennings,11854
8,Barry,Jones,7486
9,Peter,Marsh,6632


## Level Up 3: Display the names employees who have sold more than 200 different products

Hint: this is different from the previous question because the quantity sold doesn't matter, only the number of different products

In [35]:
q = """
SELECT firstName, lastName, COUNT(productCode) as different_products_sold
FROM employees JOIN customers
    ON employeeNumber = salesRepEmployeeNumber
JOIN orders USING(customerNumber)
JOIN orderdetails USING(orderNumber)
GROUP BY firstName, lastName
HAVING different_products_sold > 200
ORDER BY lastName
;
"""
pd.read_sql(q, conn)

,firstName,lastName,different_products_sold
0,Larry,Bott,236
1,Pamela,Castillo,272
2,Gerard,Hernandez,396
3,Leslie,Jennings,331
4,Barry,Jones,220
5,George,Vanauf,211


## Summary

Congrats! You practiced using join statements and leveraged your foreign keys knowledge!